In [19]:
import random

import gradio as gr
import ollama
import requests

In [36]:
MAX_HISTORY = 10

MODEL = "llama3:8b"
SYSTEM_PROMPT = (
    "You are a knowledgeable and helpful AI assistant. Provide accurate, concise, and "
    "context-aware responses tailored to the user's needs. Maintain a professional, neutral tone, "
    "ask clarifying questions if needed, and avoid speculation or harmful content. "
    "Prioritize factual accuracy, explain concepts when necessary, and adapt to the user's "
    "familiarity with the topic. Return responses in the markdown format."
)
conversation = [{"role": "system", "content": SYSTEM_PROMPT}]


def llm(MODEL, input_text):
    # Initialize the conversation if it's the first message
    global conversation

    # Append user input to the conversation
    conversation.append({"role": "user", "content": input_text})

    # Get the response from the LLM
    stream_out = ollama.chat(model=MODEL, messages=conversation, stream=True)

    # Collect the assistant's response
    output = ""
    for stream in stream_out:
        output += stream["message"]["content"]
        yield output

    # Append the assistant's response to the conversation
    conversation.append({"role": "assistant", "content": output})
    # Trim the message history if it exceeds the limit
    if len(conversation) > MAX_HISTORY:
        messages = conversation[-MAX_HISTORY:]


model_list = requests.get("http://localhost:11434/api/tags").json()["models"]
# Define the Gradio interface
interface = gr.Interface(
    fn=llm,
    inputs=[
        gr.Radio(
            choices=[(d.get("name"), d.get("model")) for d in model_list],
            value=random.choices([d.get("name") for d in model_list])[0],
            type="value",
        ),
        gr.Textbox(label="Your message:", lines=6),  # Input: User message
    ],
    outputs=[
        gr.Markdown(label="Response:", show_copy_button=True),  # Output: Assistant's response
    ],
    flagging_mode="never",
    theme=gr.Theme.from_hub("gstaff/sketch"),
    fill_width=True,
)


# Launch the Gradio interface
interface.launch(share=False)  # ,server_port=7861)

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [ ]:
# TODO : Multi-user friendly